In [14]:
import sys

sys.path.append("..")

from vq_gan_3d.model import VQGAN
from omegaconf import OmegaConf
from dataset import CombinedBrainDataset
import torch

import matplotlib.pyplot as plt

In [15]:
configure_path = "/home/sz9jt/data/t1w_processed/checkpoints/medicaldiffusion/vqgan/COMBINED_BRAIN/lightning_logs/version_60369664/hparams.yaml"
ckpt_path = "/home/sz9jt/data/t1w_processed/checkpoints/medicaldiffusion/vqgan/COMBINED_BRAIN/lightning_logs/version_60369664/checkpoints/epoch=162-step=135000-train/recon_loss=149.84.ckpt"

cfg = OmegaConf.load(configure_path)
cfg = cfg.cfg

data = torch.load(ckpt_path)
model = VQGAN(cfg)
model.load_state_dict(data["state_dict"])

test_dataset = CombinedBrainDataset(root_dir=cfg.dataset.root_dir, type="test")
test_input = test_dataset[0]["data"].unsqueeze(0)

device = torch.device("cuda:0")
model.to(device)
test_input = test_input.to(device)
model.eval()
with torch.no_grad():
    z = model.encode(test_input)
    test_output = model.decode(z)
test_output = test_output.detach().cpu().numpy().squeeze()
plt.imshow(test_output[:, :, 100], cmap="gray")